1. Indsamling og indlæsning af tekstfiler

    Kilde: New York City Year-End 2023 Enforcement Report (PDF)
    Indlæsning af PDF: Bruger jeg PyPDF2 til at udtrække tekst fra PDF-filen.

In [4]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Udtrækning af tekst fra PDF'en
nyc_report_text = extract_text_from_pdf('year-end-2023-enforcement-report.pdf')
print(nyc_report_text[:1000])  # Udskriv de første 1000 tegn af teksten for at bekræfte udtrækning


 
 
 
Crime and Enforcement Activity  
 in New York City  
(Jan 1 – Dec 31 , 2023 ) 
Edward A. Caban  
Police Commissioner   i Introduction  
 
 This report presents statistics on race/ethnicity compiled from the New York City 
Police Department’s records management system .  Crime Complaint Reports contain 
information on the crime victims’ race and ethnicity as recorded by the officer s or 
precinct clerical staff interviewing the victim or complainant.   The victim’s description of 
any non -arrested suspects will also be recorded on the crime complaint report.  Arrests 
made by the responding officers will be recorded on an arrest report a ssociated with the 
crime complaint report and used to initiate the booking system.  Once the information has 
been entered and “signed” off by a supervisor it is stored in the Department’s Enterprise 
Data Warehouse.  The data for this report were drawn from  the Data Warehouse.    
Additional information was drawn from a separate H omicide  and

Forklaring af koden og hvordan den virker

Import af PyPDF2 bibliotek: Vi importerer PyPDF2 biblioteket, som bruges til at arbejde med PDF-filer i Python.

Definere extract_text_from_pdf funktionen:
Åbning af PDF-filen: with open(pdf_path, 'rb') as file: åbner PDF-filen i binær læsetilstand.
Oprettelse af PdfReader objekt: reader = PyPDF2.PdfReader(file) opretter et PdfReader objekt, som vi bruger til at læse PDF-filen.
Løkke gennem alle sider: for page in reader.pages: itererer gennem alle sider i PDF-filen.
Udtrækning af tekst: text += page.extract_text() udtrækker tekst fra hver side og tilføjer den til text variablen.

Kalder funktionen: nyc_report_text = extract_text_from_pdf('/mnt/data/year-end-2023-enforcement-report.pdf') kalder funktionen og gemmer den udtrukne tekst i variablen nyc_report_text.

Udskrivning af tekst: print(nyc_report_text[:1000]) udskriver de første 1000 tegn af den udtrukne tekst for at bekræfte, at teksten er blevet korrekt udtrukket.

In [ ]:
Uddrag, rens og transformer tekst

Tekstrensning: Fjern unødvendige tegn, tokeniser teksten, og forbered den til NLP-behandling.

Teksttransformation: Konverter den rensede tekst til vektorer ved hjælp af TF-IDF eller word embeddings.

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Download nødvendige NLTK datafiler
nltk.download('punkt')
nltk.download('stopwords')

# Definer funktion til at rense og tokenisere tekst
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    cleaned_text = ' '.join([word for word in tokens if word.lower() not in stop_words and word.isalnum()])
    return cleaned_text

# Anvend funktionen på den udtrukne tekst fra PDF'en
cleaned_text = preprocess_text(nyc_report_text)

# Initialiser TF-IDF vektorisering og anvend det på den rensede tekst
vectorizer = TfidfVectorizer()
text_vectors = vectorizer.fit_transform([cleaned_text])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jeffo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeffo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Import af nødvendige biblioteker

Jeg importerer nltk til naturlig sprogbehandling og sklearn til tekstvektorisering.
nltk.download('punkt') og nltk.download('stopwords') sikrer, at nødvendige NLTK-datafiler er downloadet.

2. Definere funktionen preprocess_text

Funktionen tager en tekststreng som input.
Tokenisering: word_tokenize(text) splitter teksten op i individuelle ord (tokens).
Stopord: stop_words = set(stopwords.words('english')) indlæser en liste over almindelige ord (som 'the', 'and'), der fjernes fra teksten.
Rensning af tekst: Kun ord, der ikke er stopord og er alfanumeriske, bevares: [word for word in tokens if word.lower() not in stop_words and word.isalnum()].

Den rensede tekst sættes sammen igen til en enkelt streng.

3. Udførelse af tekstrensning og vektorisering

cleaned_text = preprocess_text(nyc_report_text) anvender funktionen på teksten udtrukket fra PDF'en.
vectorizer = TfidfVectorizer() initialiserer en TF-IDF vektorisering, som vægter ord baseret på deres hyppighed i dokumentet.
text_vectors = vectorizer.fit_transform([cleaned_text]) anvender TF-IDF vektorisering på den rensede tekst, hvilket resulterer i numeriske vektorer, som kan bruges til yderligere analyse.

Trin 3: Opbevaring af transformerede dokumenter:Jeg
Vi bruger numpy til at gemme og hente de transformerede vektorer.

In [8]:
import numpy as np

# Konvertering af vektorer til numpy-array
vectors = text_vectors.toarray()

# Gemme vektorerne i en fil
np.save('text_vectors.npy', vectors)

# Hente vektorerne fra filen
loaded_vectors = np.load('text_vectors.npy')

numpy er et bibliotek, der er bredt brugt til numeriske beregninger i Python. Det giver os mulighed for at gemme de transformerede tekstvektorer som filer, hvilket gør det nemt at hente og genbruge dem senere.

Anvendelse af indhold til at forbedre en forudtrænet LLM

Jeg bruger transformers biblioteket fra Hugging Face til at finjustere en forudtrænet GPT-2 model.

Jeg deler teksten op i mindre stykker for at undgå længdebegrænsningen og finjusterer modellen med disse stykker.

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Indlæsning af tokenizer og model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Forberedelse af input data
# Del teksten op i stykker på maksimalt 1024 tokens
max_length = 1024
input_texts = [cleaned_text[i:i+max_length] for i in range(0, len(cleaned_text), max_length)]

# Indstilling af model til træningstilstand
model.train()

# Træning med batch af input stykker
for input_text in input_texts:
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    loss.backward()

print("Finetuning complete.")

Finetuning complete.


Hvorfor jeg bruger det:
  Opdeling af tekstJegVi deler teksten op i stykker, så de passer til modellens maksimale sekvenslængde. Dette sikrer, ajegvi undgår længdebegrænsningsfejle
n   Batch træninJeg Vi træner modellen med hver af disse stykker for at sikre, at hele teksten bruges til finjustering.